In [1]:
from db_connection import get_postgres_connection, get_sqlserver_connection
import pandas as pd

In [7]:
def etl_item_nota_fiscal():
    """
    ETL de itens de nota fiscal
    Origem: PostgreSQL (vendas.item_nota_fiscal)
    Destino: SQL Server (dw.item_nota_fiscal)
    """

    # =====================
    # 1. EXTRACT
    # =====================
    query = """
        SELECT
            id,
            id_produto,
            id_nota_fiscal,
            quantidade,
            valor_venda_real,
            valor_unitario
        FROM vendas.item_nota_fiscal;
    """

    pg_conn = get_postgres_connection()
    df = pd.read_sql(query, pg_conn)
    pg_conn.close()

    return df

In [8]:
df = etl_item_nota_fiscal()
print(df.dtypes)

Connecting to Postgres: postgresql-datadt.alwaysdata.net datadt_digital_corporativo
id                    int64
id_produto            int64
id_nota_fiscal        int64
quantidade            int64
valor_venda_real    float64
valor_unitario      float64
dtype: object


In [10]:
# =====================
    # 2. TRANSFORM
    # =====================
df = df.copy()

df["id_produto"] = df["id_produto"].astype("int64")
df["id_nota_fiscal"] = df["id_nota_fiscal"].astype("int64")
df["quantidade"] = df["quantidade"].astype("int64")

# Remover duplicidades pela PK de origem
df = df.drop_duplicates(subset=["id"])

# Regra mínima de qualidade
df = df[df["quantidade"] > 0]

In [13]:
# =====================
# 3. LOAD
# =====================
mssql_conn = get_sqlserver_connection()
cursor = mssql_conn.cursor()

insert_sql = """
    INSERT INTO dw.item_nota_fiscal (
    id_produto, id_nota_fiscal, quantidade, valor_venda_real, valor_unitario
    )
    VALUES (?, ?, ?, ?, ?)
"""

for _, row in df.iterrows():
    cursor.execute(insert_sql, (
    row["id_produto"],
    row["id_nota_fiscal"],
    row["quantidade"],
    row["valor_venda_real"],
    row["valor_unitario"]
    ))

mssql_conn.commit()
cursor.close()
mssql_conn.close()

print("✔ Dados carregados no SQL Server")
print("🏁 ETL finalizado com sucesso")

✔ Dados carregados no SQL Server
🏁 ETL finalizado com sucesso
